# Segmentace obrazu - počítání objektů
Cvičení je zaměřené na správné využití osvětlení při nasvícení objektu a následné využití metod pro segmentaci obrazu k počítání objektů.

### Import knihoven a konfigurace

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
from pypylon import pylon 
from IPython.display import clear_output, display

%matplotlib inline

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

<a id='connect_functions'>Funkce k připojení kamery.</a>

In [ ]:
CAM_WIDTH = 1280
CAM_HEIGHT = 720

# List of features to create widgets
FEATURES = [
    {
        "name": "GainRaw",
        "type": "int"
    },
    {
        "name": "ExposureTimeRaw",
        "type": "int",
        "max": 35000,
        "min": 35,
        "step": 140,
        "value": 30124
    },
    {
        "name": "Height",
        "type": "int_text",
        "max": 1200,
        "min": 100,
        "step": "5"
    },
    {
        "name": "Width",
        "type": "int_text",
        "max": 1920,
        "min": 100,
        "step": "5"
    }
]

def connect_camera(serial_number):
    ''' Connects camera specified with its serial number
    
    Parameters
    ----------
    serial_number : string
        Camera's serial number.
    grabbed_images_path : string
        Path to folder where the saved images will be stored.
    Returns
    -------
    camera : object
    '''
    info = None
    for i in pylon.TlFactory.GetInstance().EnumerateDevices():
        if i.GetSerialNumber() == serial_number:
            info = i
            break
    else:
        print('Camera with {} serial number not found'.format(serial_number))

    # VERY IMPORTANT STEP! To use Basler PyPylon OpenCV viewer you have to call .Open() method on you camera
    if info is not None:
        camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateDevice(info)) 
        camera.Open()
        return camera
    else:
        return None    

Funkce pro zobrazení okna s přidáním eventu na klik myší.

In [ ]:
def print_xy(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONUP:
        print('x = %d, y = %d'% (x, y))      
        
def show_camera_window(img, scale=1):
    window_name = 'Camera capture (' + serial_number + ')'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow(window_name, int(img.shape[1] * scale), int(img.shape[0] * scale))
    cv2.setMouseCallback(window_name, print_xy)
    cv2.imshow(window_name, img)

<a id='preprocessing_functions'>Metody předzpracování.</a>

In [ ]:
def to_gray(img):
    ''' Converts image to monochrome
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    dst = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return dst

def to_hsv(img):
    ''' Converts image to HSV (hue, saturation, value) color space.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    dst = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    return dst

def negative(img):
    ''' Converts image to its negative
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    dst = 255 - img
    return dst

def crop(img, tl_x, tl_y, br_x, br_y):
    ''' Crops image by added coordinates
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    tl_x : int
        TOP-LEFT corner's x-coordinate
    tl_y : int
        TOP-LEFT corner's y-coordinate
    br_x : int
        BOTTOM-RIGHT corner's x-coordinate
    br_y : int
        BOTTOM-RIGHT corner's y-coordinate
    Returns
    -------
    Ouput image.
    '''
    roi = img[tl_y:br_y, tl_x:br_x]
    return roi    

<a id='segmentation_functions'>Metody segmentace.</a>

In [ ]:
def segmentation_one_threshold(img, threshold):
    '''Segments image into black & white using one threshold
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    threshold : int
        Pixels with value lower than threshold are considered black, the others white.
    Returns
    -------
    Ouput image.
    '''
    ret, dst = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    return dst

def segmentation_auto_threshold(img):
    '''Segments image into black & white using automatic threshold
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    ret, dst = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return dst

def segmentation_two_thresholds(img, lower, higher):
    '''Segments image into black & white using two thresholds
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    lower : int
        Pixels with value lower than threshold are considered black, the others white.
    higher : int
        Pixels with value higher than threshold are considered black, the others white.
    Returns
    -------
    Ouput image.
    '''
    dst = cv2.inRange(img, min(lower, higher), max(lower, higher))
    return dst

def contours(img_bin, area=0):
    '''Finds contours in binary image and filters them using their area. Then it draws binary image
    from filtered contours. It counts contours as well.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image.
    area : int
        Size of contour that is used to filter all smaller contours out.
    Returns
    -------
    contour_drawn : numpy.ndarray
        Output binary image with drawn filled filtered contours.
    count : int
        Number of found and filtered contours.
    '''
    _, contours, _  = cv2.findContours(img_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours =  [c for c in contours if cv2.contourArea(c) > area]
    contour_drawn = cv2.drawContours(np.zeros(img_bin.shape, dtype=np.uint8), contours, -1, color=(255, 255, 255), thickness=cv2.FILLED)
    return contour_drawn, len(contours)

def mask(img, mask_bin):
    '''Masks colored image with binary mask. Output image is just logical AND between two images.'''
    return cv2.bitwise_and(img, img, mask = mask_bin) 

<a id='filtration_functions'>Metody filtrace.</a>

In [ ]:
def filtration_median(img, filter_size):
    '''Filters image noise using median algorithm
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    filter_size : int
        Size of median filter.
    Returns
    -------
    Ouput image.
    '''
    dst = cv2.medianBlur(img, filter_size)
    return dst    

<a id='show_functions'>Metoda pro zobrazení různého množství obrázků.</a>

In [ ]:
def show_images(*imgs, scale=1, window_name='Image preview'):
    """ Opens multiple image previews depending on the length of the input *imgs list.
    The preview is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    *imgs : list
        Multiple input images which have to be shown.
    scale : double
        Scale of shown image window.
    window_name : Optional[string]
        An optional window name.
    Returns
    -------
    None
    """
    for i, img in enumerate(imgs, 1):
        window_name_id = window_name + ' ' + str(i)
        cv2.namedWindow(window_name_id, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
        cv2.resizeWindow(window_name_id, img.shape[1] * scale, img.shape[0] * scale)
        cv2.moveWindow(window_name_id, 0, i*10)

    while 1:
        for i, img in enumerate(imgs, 1):
            cv2.imshow(window_name + ' ' + str(i), img)
            
        k = cv2.waitKey(0)

        if k == ord('q'):
            break

    cv2.destroyAllWindows()
    
def plot_image(img):
    '''Plots image in matplotlib inline window for demonstration.'''
    plt.axis("off")
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

<a id='others'>Metody ostatní</a>

In [ ]:
def to_intensity(hue_angle):
    '''Converts color angle in HUE definition into intensity value of brightness image in opencv
    
    Parameters
    ----------
    hue_angle : int
        Angle in HUE definition.
    Returns
    -------
    Integer value that represents the same HUE value but in opencv brightness image.
    '''
    return int(hue_angle * 0.5)

---

Seznam funkcí pro přehlednost:
- [`connect_camera(...)`](#connect_functions)
- [`show_images(...)`](#show_functions)


- [`to_gray(...)`](#preprocessing_functions)
- [`to_hsv(...)`](#preprocessing_functions)
- [`negative(...)`](#preprocessing_functions)
- [`crop(...)`](#preprocessing_functions)


- [`segmentation_one_threshold(...)`](#segmentation_functions)
- [`segmentation_auto_threshold(...)`](#segmentation_functions)
- [`segmentation_two_thresholds(...)`](#segmentation_functions)
- [`contours(...)`](#segmentation_functions)


- [`filtration_median(...)`](#filtration_functions)


- [`to_intensity(...)`](#others)

### Úkol
Zvolte vhodné funkce pro segmentaci obrazu a spočítejte kolik se na obrázku vyskytuje celkem objektů. Využijte k tomu znalosti o segmentaci objektů podle barvy (*HINT: jiné barevné spektrum než RGB*) a postupně spočtěte objekty stejných barev, které nakonec sečtete dohromady.

Pro testování algoritmu využijte funkci `algorithm()`. Funkce je vložena do snímání kamery. Je tak možné v reálném čase pozorovat, co se s obrazem děje. Po změně obsahu funkce je však potřeba načíst znovu i blok pro spuštění snímání z kamery.

Po vyladění algoritmu na jednotlivé úlohy, demonstrujte každý z nich na 1 získaném obrázku.

#### 1) Nastavte správně kameru.

In [ ]:
serial_number = '' ### vlastní sériové číslo kamery
width = 1280
height = 720

camera = ...(serial_number) ###

#### 2) Doplňujte testovací verzi algoritmu.

In [ ]:
def algorithm(img):
    ### 
    
    
    
    # Zde je potřeba vytvořit binární obrázek
    img_bin = ###
    
    # Výsledný obrázek bude výběr z původního pomocí binární masky
    res = mask(img, img_bin)
    return res

Definuje obsah funkce pro zpracování obrazu, konkrétně jeho segmentaci. Funkce `impro` je **neměnná**.

In [ ]:
def impro(img):
    res = algorithm(img)

    show_camera_window(res, 1)
    return res

#### 3) Spusťte snímání a testujte algoritmus pro zpracování obrazu.

Info k oknu:
- Pro spuštění okna kamery je zapotřebí kliknout na tlačítko `Run Interact`. 
- Okno kamery se vypne stisknutím tlačítka `q`.
- Obrázek se uloží stisknutím tlačítka `s`.
- Po kliku levého tlačítka myši se vytisknou souřadnice místa v obraze.
- Pro projevení změn v nastavení pomocí GUI prvků je nutné ukončit okno kamery a znovu ho spustit.

In [ ]:
from pypylon_opencv_viewer import BaslerOpenCVViewer
    
viewer = BaslerOpenCVViewer(camera)
viewer.set_features(FEATURES)
viewer.set_impro_function(impro)
viewer.run_interaction_continuous_shot(window_size=(width, height))

#### 4) Doplňte algoritmus pro segmentaci objektů podle jednotlivých barev. 
Vytvořte si konkrétní prahy, které budou na danou barvu stačit. Tolerance je jedno číslo. Funkce `count_all_objects` je **neměnná**.

In [ ]:
thresholds = [
    ### 
]
tolerance = ###

def algorithm_color(img, thresh, tolerance):
    ### Potřeba doplnit celý algoritmus
    
    
    
    
    
    
    

    # Zde je potřeba doplnit binární obrázek
    img_bin =  ###
    
    # Zobrazení obrázku v plotu (maskování původního obrázku binární maskou)
    plot_image(mask(img, img_bin))
    return count
    

def count_all_objects(img):
    sum_all = 0
    for thresh in thresholds:
        sum_all += algorithm_color(img, thresh, tolerance)
        
    return sum_all

#### 5) Spusťte výsledný algoritmus pro spočtení objektův obraze. 

In [ ]:
image_path = '' ###
image = cv2.imread(image_path)
count = count_all_objects(image)
print('Celkem je v obraze ' + str(count) + ' objektů.')